In [1]:
import pandas as pd
import os
from data_processing import steps, data, data_train, data_test, exog_vars, hour_in_week

cwd = os.path.abspath("")
backtest_result_file_path = os.path.join(cwd, '../data/backtest_result.csv')

df = pd.read_csv(backtest_result_file_path)

c:\Users\alfon\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total Load
Most recent forecast
Most recent P10
Most recent P90
Day-ahead 6PM forecast
Day-ahead 6PM P10
Day-ahead 6PM P90
Week-ahead forecast
Train dates : 2022-05-25 00:00:00+00:00 --- 2023-04-20 23:00:00+00:00  (n=7944)
Test dates  : 2023-04-21 00:00:00+00:00 --- 2023-06-15 23:00:00+00:00  (n=1344)


In [13]:
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from sklearn.ensemble import RandomForestRegressor

# Load your data
url = 'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv'
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]

# Split data into training and backtesting sets
n_backtest = 36 * 3  # Last 9 years for backtesting
data_train = data[:-n_backtest]
data_backtest = data[-n_backtest:]

# Initialize the forecaster
forecaster = ForecasterAutoreg(
    regressor=RandomForestRegressor(random_state=123),
    lags=15
)

# Perform backtesting
metric, predictions_backtest = backtesting_forecaster(
    forecaster=forecaster,
    y=data['y'],
    initial_train_size=len(data_train),
    steps=10,
    metric='mean_squared_error',
    refit=True,
    verbose=True
)

# Print fold information
print(f"Number of folds: {len(predictions_backtest)}")
for i, fold in enumerate(predictions_backtest):
    print(f"Fold {i}: {fold}")


Information of backtesting process
----------------------------------
Number of observations used for initial training: 96
Number of observations used for backtesting: 108
    Number of folds: 11
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 8 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 1999-06-01 00:00:00  (n=96)
    Validation: 1999-07-01 00:00:00 -- 2000-04-01 00:00:00  (n=10)
Fold: 1
    Training:   1992-05-01 00:00:00 -- 2000-04-01 00:00:00  (n=96)
    Validation: 2000-05-01 00:00:00 -- 2001-02-01 00:00:00  (n=10)
Fold: 2
    Training:   1993-03-01 00:00:00 -- 2001-02-01 00:00:00  (n=96)
    Validation: 2001-03-01 00:00:00 -- 2001-12-01 00:00:00  (n=10)
Fold: 3
    Training:   1994-01-01 00:00:00 -- 2001-12-01 00:00:00  (n=96)
    Validation: 2002-01-01 00:00:00 -- 2002-10-01 00:00:00  (n=10)
Fold: 4
    Training:   1994-11-01 00:00:00 -- 2002-10-01 00:00:00  (n=96)


100%|██████████| 11/11 [00:00<00:00, 196.73it/s]


Number of folds: 108
Fold 0: pred


In [1]:
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from sklearn.ensemble import RandomForestRegressor
from forecast import BacktestingForecaster

# Load your data
url = 'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv'
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]

# Split data into training and backtesting sets
n_backtest = 36 * 3  # Last 9 years for backtesting
data_train = data[:-n_backtest]
data_backtest = data[-n_backtest:]

# Initialize the forecaster
forecaster = ForecasterAutoreg(
    regressor=RandomForestRegressor(random_state=123),
    lags=15
)
bf = BacktestingForecaster(
    forecaster=forecaster,
    y=data['y'],
    initial_train_size=len(data_train),
    steps=10,
    metric='mean_squared_error',
)


c:\Users\alfon\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fold_training_index, fold_validation_index = bf._create_backtesting_folds()

In [3]:
# Perform backtesting
metric, predictions_backtest = backtesting_forecaster(
    forecaster=forecaster,
    y=data['y'],
    initial_train_size=len(data_train),
    steps=10,
    metric='mean_squared_error',
    refit=True,
    verbose=True
)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 96
Number of observations used for backtesting: 108
    Number of folds: 11
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 8 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 1999-06-01 00:00:00  (n=96)
    Validation: 1999-07-01 00:00:00 -- 2000-04-01 00:00:00  (n=10)
Fold: 1
    Training:   1992-05-01 00:00:00 -- 2000-04-01 00:00:00  (n=96)
    Validation: 2000-05-01 00:00:00 -- 2001-02-01 00:00:00  (n=10)
Fold: 2
    Training:   1993-03-01 00:00:00 -- 2001-02-01 00:00:00  (n=96)
    Validation: 2001-03-01 00:00:00 -- 2001-12-01 00:00:00  (n=10)
Fold: 3
    Training:   1994-01-01 00:00:00 -- 2001-12-01 00:00:00  (n=96)
    Validation: 2002-01-01 00:00:00 -- 2002-10-01 00:00:00  (n=10)
Fold: 4
    Training:   1994-11-01 00:00:00 -- 2002-10-01 00:00:00  (n=96)


100%|██████████| 11/11 [00:00<00:00, 300.68it/s]


In [41]:
"""
   folds = _create_backtesting_folds(
                data                  = y,
                window_size           = window_size,
                initial_train_size    = initial_train_size,
                test_size             = steps,
                externally_fitted     = externally_fitted,
                refit                 = refit,
                fixed_train_size      = fixed_train_size,
                gap                   = gap,
                skip_folds            = skip_folds,
                allow_incomplete_fold = allow_incomplete_fold,
                return_all_indexes    = False,
                differentiation       = differentiation,
                verbose               = verbose
                forecaster.window_size_diff

"""

'\n   folds = _create_backtesting_folds(\n                data                  = y,\n                window_size           = window_size,\n                initial_train_size    = initial_train_size,\n                test_size             = steps,\n                externally_fitted     = externally_fitted,\n                refit                 = refit,\n                fixed_train_size      = fixed_train_size,\n                gap                   = gap,\n                skip_folds            = skip_folds,\n                allow_incomplete_fold = allow_incomplete_fold,\n                return_all_indexes    = False,\n                differentiation       = differentiation,\n                verbose               = verbose\n\n'

In [46]:
from skforecast.model_selection import _create_backtesting_folds

backtesting_folds = _create_backtesting_folds(
    data =data['y'],
    initial_train_size=len(data_train),
    refit=True,
    test_size= 10,
    window_size=forecaster.window_size_diff,

)



Information of backtesting process
----------------------------------
Number of observations used for initial training: 96
Number of observations used for backtesting: 108
    Number of folds: 11
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 8 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 1999-06-01 00:00:00  (n=96)
    Validation: 1999-07-01 00:00:00 -- 2000-04-01 00:00:00  (n=10)
Fold: 1
    Training:   1992-05-01 00:00:00 -- 2000-04-01 00:00:00  (n=96)
    Validation: 2000-05-01 00:00:00 -- 2001-02-01 00:00:00  (n=10)
Fold: 2
    Training:   1993-03-01 00:00:00 -- 2001-02-01 00:00:00  (n=96)
    Validation: 2001-03-01 00:00:00 -- 2001-12-01 00:00:00  (n=10)
Fold: 3
    Training:   1994-01-01 00:00:00 -- 2001-12-01 00:00:00  (n=96)
    Validation: 2002-01-01 00:00:00 -- 2002-10-01 00:00:00  (n=10)
Fold: 4
    Training:   1994-11-01 00:00:00 -- 2002-10-01 00:00:00  (n=96)


In [48]:
len(backtesting_folds)

11